In [1]:
import pandas as pd
import numpy as np
import io 
import os 
import json 
import distutils.dir_util
from collections import Counter
from tensorflow.keras.models import save_model 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout
#import tensorflow as tf

In [2]:
#import keras.backend.tensorflow_backend

In [4]:
#os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [5]:
#from keras.utils import multi_gpu_model

In [6]:
#from keras.backend import tensorflow_backend as K

In [7]:
#from tensorflow.python.client import device_lib ## 필요한 패키지 불러오기 ##
#import keras.backend.tensorflow_backend as K ## 필요한 패키지 불러오기 ##

In [8]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [9]:
def load_train_json(fname):
    f = pd.read_json(fname+'.json' ,typ = 'frame', encoding="utf-8")
    df = pd.DataFrame(f)
    df = df.sort_values(by=['like_cnt'],ascending=False)
    df = df[df['like_cnt']>10]
    print('load_train_json')
    return df

In [10]:
def load_val_json(fname):
    f = pd.read_json(fname+'.json' ,typ = 'frame', encoding="utf-8")
    df = pd.DataFrame(f)
    df = df.sort_values(by=['like_cnt'],ascending=False)
    #df = df[df['like_cnt']>10]
    print('load_train_json')
    return df

In [11]:
def most_popular(df,song_num):
    train_song = df['songs']
    train_tag = df['tags']
    
    song_list = [song for plist in train_song for song in plist ]
    tag_list = [tag for plist in train_tag for tag in plist]
    
    count_song = Counter(song_list)
    count_tag = Counter(tag_list)
    x={}
    for key, value in count_song.items():
        if value>song_num:
            x[key]=value
    song_len = len(x)
    for key, value in count_tag.items():
        if value>7:
            x[key]=value
    tag_len=len(x)-song_len
    print('most_popular')
    return list(x.keys()),song_len, tag_len

In [12]:
def create_zero(column_name):
    zero_df=pd.DataFrame(columns=col)
    return zero_df

In [13]:
def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for tag,song in zip(df.iloc[i,0],df.iloc[i,3]):
            if tag in column_name:
                zero_df.iloc[i,column_name.index(tag)]=1
            if song in column_name:
                zero_df.iloc[i,column_name.index(song)]=1
    return zero_df

In [28]:
def deep_learing(column_name,train_onehot,val_onehot):
        col = column_name
        encoding_dim=64

        input_plist=Input(shape=(len(col),))
        dropout = Dropout(0.2)(input_plist)
        
        encoded=Dense(32,activation='relu')(input_plist)
        
        decoded=Dense(len(col),activation='sigmoid')(encoded)
        
        autoencoder=Model(input_plist,decoded)
    
        autoencoder.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        
        autoencoder.fit(train_onehot,train_onehot,epochs=50,batch_size=32,validation_split=0.1)
        
        predict_plist=autoencoder.predict(val_onehot)
        print('deep_learning')
        return predict_plist


In [16]:
def result(df_id, column_name, song_len, tag_len, predict):
        #train_df = df
        df_id = list(df_id)
        col= column_name
        ori_song = col[:song_len]
        ori_tag = col[song_len:]
        #onehot = df_one
        #predict = deep_learning(train)
        print('ok')
        song_predict = predict[:,:song_len]
        tag_predict = predict[:,song_len:]
        print('ok')
        result=[]
        n=0
        for i in df_id:
            dic={}
            dic['id']=i

            plist_song=song_predict[n].argsort()[-100:]
            p_song=[]
            for song in plist_song:
                p_song.append(ori_song[song])
            dic['songs']=p_song

            plist_tag=tag_predict[n].argsort()[-10:]
            p_tag=[]
            for tag in plist_tag:
                p_tag.append(ori_tag[tag])
            dic['tags']=p_tag
            n+=1
            result.append(dic)
        print('result')
        return result


In [17]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath( parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


In [18]:
train_df=load_train_json('data/train')
t_col,t_song, t_tag=most_popular(train_df,15)
print(len(t_col))

load_train_json
most_popular
33781


In [19]:
t_song

31730

In [21]:
train_one = create_onehot(train_df, t_col)

In [22]:
train_one.shape

(47585, 33781)

In [23]:
val_df=load_val_json('data/val')
val_one=create_onehot(val_df, t_col)

load_train_json


In [24]:
val_one.shape

(23015, 33781)

In [29]:
predict = deep_learing(t_col,train_one, val_one)

Epoch 1/50
1339/1339 [==============================] - 54s 40ms/step - loss: 0.0600 - accuracy: 0.0120 - val_loss: 0.0030 - val_accuracy: 0.0124
Epoch 2/50
1339/1339 [==============================] - 57s 43ms/step - loss: 0.0025 - accuracy: 0.0123 - val_loss: 0.0021 - val_accuracy: 0.0124
Epoch 3/50
1339/1339 [==============================] - 50s 37ms/step - loss: 0.0020 - accuracy: 0.0123 - val_loss: 0.0018 - val_accuracy: 0.0124
Epoch 4/50
1339/1339 [==============================] - 52s 38ms/step - loss: 0.0018 - accuracy: 0.0123 - val_loss: 0.0016 - val_accuracy: 0.0124
Epoch 5/50
1339/1339 [==============================] - 52s 39ms/step - loss: 0.0016 - accuracy: 0.0152 - val_loss: 0.0015 - val_accuracy: 0.0237
Epoch 6/50
1339/1339 [==============================] - 54s 40ms/step - loss: 0.0014 - accuracy: 0.0368 - val_loss: 0.0013 - val_accuracy: 0.0672
Epoch 7/50
1339/1339 [==============================] - 51s 38ms/step - loss: 0.0013 - accuracy: 0.0620 - val_loss: 0.0012 -

In [30]:
result=result(val_df['id'],t_col, t_song, t_tag, predict )

ok
ok
result


In [31]:
write_json(result,'denoising/results2')

In [32]:
result = pd.read_json('denoising/results2.json')

In [33]:
result2 = pd.DataFrame(data=result)

In [34]:
result2.sort_values(by=['id'], axis=0)

,id,songs,tags
5734,3,"[321296, 469774, 547386, 663256, 355518, 18213...","[졸릴때, 인생곡, 신나는곡, 운전, 신나는노래, 여름노래, 차안에서, 추천곡, 신..."
21172,10,"[261853, 146449, 520522, 603475, 594905, 41800...","[함께, 프로야구, 감성음악, 동화, 편안함, 좋은, 신화, 나이키, 서태지, 아기동요]"
21852,18,"[637303, 420490, 546490, 634669, 416859, 28588...","[시원한, 노래추천, 여름향기, 방학, 무더위, 시원한음악, 청량, 더위, 시원, 여름]"
11432,22,"[103530, 300651, 85316, 574119, 628248, 611706...","[신화, 아련, 프로야구, K팝스타, 좋은, 멜로디, 노래추천, 서태지, 나이키, ..."
10694,23,"[103530, 300651, 85316, 574119, 628248, 611706...","[신화, 아련, 프로야구, K팝스타, 좋은, 멜로디, 노래추천, 서태지, 나이키, ..."
...,...,...,...
22009,153382,"[367874, 412695, 463466, 574856, 474353, 43714...","[낮잠, 봄날, 생각, 자장가, 가사없는, 피아노, 숙면, 연주곡, OST, 뉴에이지]"
11632,153386,"[645130, 15434, 333461, 429601, 76453, 40937, ...","[느낌, 소울, 여자보컬, 목소리, 노래추천, 감미로운, 꿀음색, 음색깡패, 음색,..."
830,153411,"[367260, 449338, 547386, 264002, 584191, 63553...","[까페, 낭만적인, 휴식힐링, 젝스키스, 끈적한, 신화, 명곡모음, 추억, 복면가왕..."
3756,153417,"[103530, 300651, 85316, 574119, 628248, 611706...","[신화, 아련, 프로야구, K팝스타, 좋은, 멜로디, 노래추천, 서태지, 나이키, ..."


In [35]:
val = pd.read_json('data/val.json')

In [36]:
val.sort_values(by=['id'], axis=0)

,tags,id,plylst_title,songs,like_cnt,updt_date
20455,[드라이브할때신나는노래],3,,"[174104, 377243, 442123, 590253, 372171, 14281...",24,2016-12-12 09:59:28.000
21977,[재즈추천],10,,"[477623, 571909, 232780, 22855, 572565, 696141...",0,2019-03-27 15:20:35.000
6229,[여름],18,,"[630498, 157520, 116862, 204839, 169945, 25412...",0,2017-04-13 19:14:28.000
5102,[],22,추억속에 민들레 되어...,[],7,2011-05-09 19:05:21.000
11188,[],23,,"[606923, 364111, 538777, 234196, 707031, 63575...",8,2014-03-21 09:59:27.000
...,...,...,...,...,...,...
12683,"[태교음악, 감성자극, OST, 숙면, 뉴에이지]",153382,,"[437087, 188137, 608545, 156468, 20735, 46911,...",0,2019-04-23 11:58:31.000
18472,"[알앤비, 음색]",153386,,"[224176, 457916, 430800, 349046, 647312, 59677...",7,2020-01-22 02:46:02.000
10693,"[oldpop, 회상]",153411,,"[113088, 55264, 209280, 562707, 237235, 661575...",523,2018-04-22 15:43:58.000
7222,[],153417,아침에 일어나 눈 비비며 듣는 CCM,[],52,2017-04-23 07:06:41.000


In [37]:
#train_one.to_csv('denoising/train_onehot.csv',index = False) 
#val_one.to_csv('denoising/val_onehot.csv', index = False)